# 第3章: 正規表現

Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．

- 1行に1記事の情報がJSON形式で格納される
- 各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
- ファイル全体はgzipで圧縮される

以下の処理を行うプログラムを作成せよ．

In [2]:
import os
import re
import json
import gzip
import numpy as np

In [3]:
eng_text = open(os.path.join('data', 'eng.txt')).read()

In [3]:
!mkdir ./data
!curl -O https://nlp100.github.io/data/jawiki-country.json.gz
!mv  jawiki-country.json.gz ./data
!ls data/

mkdir: ./data: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4949k  100 4949k    0     0  8950k      0 --:--:-- --:--:-- --:--:-- 8950k
jawiki-country.json.gz


## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [8]:
display_characters = 1000 ## 1000文字だけ表示
filepath = os.path.join('data', 'jawiki-country.json.gz')
eng_text = None
with gzip.open(filepath, 'r') as f:
    for line in f:
        obj = json.loads(line)
        if 'イギリス' == obj['title']:
            print(obj['text'][:display_characters])
            eng_text = obj['text']

eng_text_filepath = os.path.join('data', 'eng.txt')
with open(eng_text_filepath, 'w') as f:
    f.write(eng_text)

{{redirect|UK}}
{{redirect|英国|春秋時代の諸侯国|英 (春秋)}}
{{Otheruses|ヨーロッパの国|長崎県・熊本県の郷土料理|いぎりす}}
{{基礎情報 国
|略名  =イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
|国旗画像 = Flag of the United Kingdom.svg
|国章画像 = [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
|国章リンク =（[[イギリスの国章|国章]]）
|標語 = {{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Die

## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [4]:
for line in eng_text.split('\n'):
    if 'Category' in line:
        print(line)

{{Sisterlinks|commons=United Kingdom|commonscat=United Kingdom|s=Category:イギリス|n=Category:イギリス|voy=United Kingdom}}
[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]


## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [5]:
import re
categories = re.findall('Category: ?([^| \]]+)', eng_text)
for c in sorted(set(categories), key=len):
    print(c)

島国
イギリス
海洋国家
G8加盟国
英連邦王国
現存する君主国
欧州連合加盟国
イギリス連邦加盟国
1801年に成立した国家・領域


## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [6]:
url_replaced_txt = re.sub('https?://[\w!?/\+\-_~=;\.,*&@#$%\(\)\'\[\]]+', 'URL', eng_text)
candidates = re.findall('(=+)([^| \]=}:&]+)(=+)', url_replaced_txt) ## &を除外するのは不本意

sections = []
for prefix, section_name, suffix in candidates:
    if len(prefix) != len(suffix):
        continue
    level = len(prefix)
    sections.append((section_name, level))
for section, level in sorted(sections, key=lambda x: x[1]):
    print(f"{section:^5}: {level:>1}")

 国名  : 2
 歴史  : 2
 地理  : 2
 政治  : 2
 経済  : 2
 交通  : 2
科学技術 : 2
 国民  : 2
 文化  : 2
 脚注  : 2
関連項目 : 2
外部リンク: 2
主要都市 : 3
 気候  : 3
 元首  : 3
  法  : 3
 内政  : 3
地方行政区分: 3
外交・軍事: 3
 鉱業  : 3
 農業  : 3
 貿易  : 3
 不動産 : 3
エネルギー政策: 3
 通貨  : 3
 企業  : 3
 道路  : 3
 鉄道  : 3
 海運  : 3
 航空  : 3
 言語  : 3
 宗教  : 3
 婚姻  : 3
 移住  : 3
 教育  : 3
 医療  : 3
 食文化 : 3
 文学  : 3
 哲学  : 3
 音楽  : 3
 映画  : 3
コメディ : 3
 国花  : 3
世界遺産 : 3
 祝祭日 : 3
スポーツ : 3
 通信  : 4
ポピュラー音楽: 4
サッカー : 4
クリケット: 4
 競馬  : 4
モータースポーツ: 4
 野球  : 4


## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [7]:
re.findall('https?://[\w!?/\+\-_~=;\.,*&@#$%\(\)\'\[\]]+', eng_text)
# 'http://webarchive.nationalarchives.gov.uk/20121015000000/http' のようなものも取れてしまう。。

['https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates',
 'http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a=IMF',
 'http://webarchive.nationalarchives.gov.uk/+/http',
 'http://warp.da.ndl.go.jp/info',
 'https://kotobank.jp/word/%E3%82%A8%E3%82%B2%E3%83%AC%E3%82%B9-444373',
 'http://www.scotshistoryonline.co.uk/union.html',
 'http://books.google.com/?id=LYc1tSYonrQC&pg=PA165',
 'http://www.historyworld.net/wrldhis/PlainTextHistories.asp?historyid=ab07',
 'http://www.iso.org/iso/iso_3166-2_newsletter_ii-3_2011-12-13.pdf',
 'http://books.google.com/?id=gPkDAQAAIAAJ',
 'http://books.google.com/?id=2u8rD6F-yg0C&pg=PA7',
 'http://www.ons.gov.uk/ons/dcp171778_346117.xml',
 'https://web.archive.org/web/20141224045523/http',
 'http://www.gmb.org.uk/newsroom/fall-in-earnings-value-during-recession',
 'http://www.merriam-we

## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [8]:
pattern = '\|([^=]+)=[ \[（]*([^\]]+)'
print(re.search(pattern, '|他元首等肩書1 = [[貴族院 (イギリス)|貴族院議長]]').groups())
print(re.search(pattern, '|略名  =イギリス').groups())
print(re.search(pattern, '|国章リンク =（[[イギリスの国章|国章]]）').groups())

('他元首等肩書1 ', '貴族院 (イギリス)|貴族院議長')
('略名  ', 'イギリス')
('国章リンク ', 'イギリスの国章|国章')


In [9]:
# 基礎情報 原文
double_curly_count = 0
basic_info = []
for line in eng_text.split('\n'):
    if '{{基礎情報' in line:
        double_curly_count = 1
        basic_info.append(line)
    elif double_curly_count > 0:
        basic_info.append(line)
        if '{{' in line or '}}' in line:
            if '{{' in line:
                double_curly_count += 1
            if '}}' in line:
                double_curly_count -= 1
        if double_curly_count == 0:
            break
print('\n'.join(basic_info))

{{基礎情報 国
|略名  =イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
|国旗画像 = Flag of the United Kingdom.svg
|国章画像 = [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
|国章リンク =（[[イギリスの国章|国章]]）
|標語 = {{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）
|国歌 = [[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br /

In [10]:
# 基礎情報をparse
fields_dict = {}
basic_info_pattern = '\|(.+)=(.+)'
for info_line in basic_info:
    m = re.search(basic_info_pattern, info_line)
    if m:
        field, val = m.groups()
        fields_dict[field.strip()] = val.strip()
fields_dict

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）',
 '国歌': "[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />''神よ女王を護り賜え''<br />{{center|[[ファイル:United States Navy Band - God Save the Queen.ogg]]}}",
 '地図画像': 'Europe-UK.svg',
 '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg',
 '公用語': '[[英語]]',
 '首都': '[[ロンドン]]（事実上）',
 '最大都市': 'ロンドン',
 '元首等肩書': '[[イギリスの君主|女王]]',
 '元首等氏名': '[[エリザベス2世]]',
 '首相等肩書': '[[イギリスの首相|首相]]',
 '首相等氏名': '[[ボリス・ジョンソン]]',
 '他元首等肩書1': '[[貴族院 (イギリス)|貴族院議長]]',
 '他元首等氏名1': '[[:en:Norman Fowler, Baron Fowler|ノーマン・ファウラー]]',
 '他元首等肩書2': '[[庶民院 (イギリス)|庶民院議長]]',
 '他元首等氏名2': '{{仮リンク|リンゼイ・ホイル|en|Linds

## 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: [マークアップ早見表](https://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [11]:
re.sub("'?''", '', "|確立形態4 = 現在の国号「'''グレートブリテン及び北アイルランド連合王国'''」に変更")

'|確立形態4 = 現在の国号「グレートブリテン及び北アイルランド連合王国」に変更'

In [12]:
# helper
def remove_pattern(pattern, line, split_delim=None):
    m = re.search(pattern, line)
    if m:
        val, = m.groups()
        if split_delim:
            val = val.split(split_delim)[-1]
        return val
    return line

In [13]:
target_keys = []
for key, val in fields_dict.items():
    if "''" in val:
        target_keys.append(key)
        print(key,val)

国歌 [[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />''神よ女王を護り賜え''<br />{{center|[[ファイル:United States Navy Band - God Save the Queen.ogg]]}}
確立形態4 現在の国号「'''グレートブリテン及び北アイルランド連合王国'''」に変更


In [14]:
# 基礎情報をparse
fields_dict = {}
basic_info_pattern = '\|(.+)=(.+)'
for info_line in basic_info:
    # 強調マークアップを削除
    flat_line = re.sub("'?''", '', info_line)
    m = re.search(basic_info_pattern, flat_line)
    if m:
        field, val = m.groups()
        fields_dict[field.strip()] = val.strip()
for _k in target_keys:
    print(_k, fields_dict[_k])

国歌 [[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />神よ女王を護り賜え<br />{{center|[[ファイル:United States Navy Band - God Save the Queen.ogg]]}}
確立形態4 現在の国号「グレートブリテン及び北アイルランド連合王国」に変更


## 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: [マークアップ早見表](https://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [17]:
target_keys = []
for key, val in fields_dict.items():
    m = re.search(r'\[\[.+\]\]', val)
    if m:
        target_keys.append(key)
        print(key, val)

国章画像 [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
国章リンク （[[イギリスの国章|国章]]）
標語 {{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）
国歌 [[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />神よ女王を護り賜え<br />{{center|[[ファイル:United States Navy Band - God Save the Queen.ogg]]}}
公用語 [[英語]]
首都 [[ロンドン]]（事実上）
元首等肩書 [[イギリスの君主|女王]]
元首等氏名 [[エリザベス2世]]
首相等肩書 [[イギリスの首相|首相]]
首相等氏名 [[ボリス・ジョンソン]]
他元首等肩書1 [[貴族院 (イギリス)|貴族院議長]]
他元首等氏名1 [[:en:Norman Fowler, Baron Fowler|ノーマン・ファウラー]]
他元首等肩書2 [[庶民院 (イギリス)|庶民院議長]]
他元首等肩書3 [[連合王国最高裁判所|最高裁判所長官]]
他元首等氏名3 [[:en:Brenda Hale, Baroness Hale of Richmond|ブレンダ・ヘイル]]
確立形態1 [[イングランド王国]]／[[スコットランド王国]]<br />（両国とも[[合同法 (1707年)|1707年合同法]]まで）
確立形態2 [[グレートブリテン王国]]成立<br />（1707年合同法）
確立形態3 [[グレートブリテン及びアイルランド連合王国]]成立<br />（[[合同法 (1800年)|1800年合同法]]）
通貨 [[スターリング・ポンド|UKポンド]] (£)
ccTLD [[.uk]] / [[.gb]]<ref>使用は.ukに比べ圧倒的少数。</ref>


In [110]:
def replace_internal_markup(string):
    pattern = r'(?:(\[\[[^\|\[\]]+\]\]))|(?:(\[\[ファイル:[^\]]+\]\]))|(?:(\[\[[^\[\]]+\]\]))'
    matches = re.findall(pattern, string)
    ret_str = string
    for match_word, file_markup, match_with_pipe in matches:
        if match_word:
            norm_word = re.sub(r'\[\[(.+)\]\]', r'\1', match_word)
            ret_str = ret_str.replace(match_word, norm_word)
        elif match_with_pipe:
            norm_word = re.sub(r'\[\[[^\|]+\|([^\]]+)\]\]', r'\1', match_with_pipe)
            ret_str = ret_str.replace(match_with_pipe, norm_word)
        elif file_markup:
            norm_word = file_markup.split('|')[-1].replace(']]', '')
            ret_str = ret_str.replace(file_markup, norm_word)
    return ret_str
replace_internal_markup('[[aaa]]abc[[bbb]][[Dieu et mon droit|神と我が権利]][[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]')

'aaaabcbbb神と我が権利イギリスの国章'

In [111]:
fields_dict = {}
basic_info_pattern = '\|(.+)=(.+)'
for info_line in basic_info:
    # 強調マークアップを削除
    flat_line = re.sub("'?''", '', info_line)
    m = re.search(basic_info_pattern, flat_line)
    if m:
        field, val = m.groups()
        # 内部リンクマークアップを除去
        val = replace_internal_markup(val)
        fields_dict[field.strip()] = val.strip()
fields_dict

for _k in target_keys:
    print(_k, fields_dict[_k])

国章画像 イギリスの国章
国章リンク （国章）
標語 {{lang|fr|Dieu et mon droit}}<br />（フランス語:神と我が権利）
国歌 {{lang|en|God Save the Queen}}{{en icon}}<br />神よ女王を護り賜え<br />{{center|ファイル:United States Navy Band - God Save the Queen.ogg}}
公用語 英語
首都 ロンドン（事実上）
元首等肩書 女王
元首等氏名 エリザベス2世
首相等肩書 首相
首相等氏名 ボリス・ジョンソン
他元首等肩書1 貴族院議長
他元首等氏名1 ノーマン・ファウラー
他元首等肩書2 庶民院議長
他元首等肩書3 最高裁判所長官
他元首等氏名3 ブレンダ・ヘイル
確立形態1 イングランド王国／スコットランド王国<br />（両国とも1707年合同法まで）
確立形態2 グレートブリテン王国成立<br />（1707年合同法）
確立形態3 グレートブリテン及びアイルランド連合王国成立<br />（1800年合同法）
通貨 UKポンド (£)
ccTLD .uk / .gb<ref>使用は.ukに比べ圧倒的少数。</ref>


## 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ

In [158]:
def replace_curly(string):
    pattern = r'({{lang\|[^\|]+\|[^}]+}})|({{[^\|]+}})|({{[^\|]+\|ファイル:[^}]+}})'
    matches = re.findall(pattern, string)
    ret_str = string
    for lang_markup, icon_markup, file_markup in matches:
        if lang_markup:
            norm_word = re.sub(r'{{lang\|[^\|]+\|([^}]+)}}', r'\1', lang_markup)
            ret_str = ret_str.replace(lang_markup, norm_word)
        elif icon_markup:
            #             norm_word = re.sub(r'{{([^\|]+)}}', r'\1', icon_markup)
            ret_str = ret_str.replace(icon_markup, '')
        elif file_markup:
            norm_word = re.sub(r'{{[^\|]+\|ファイル:([^}]+)}}', r'\1', file_markup)
            ret_str = ret_str.replace(file_markup, norm_word)
    return ret_str    

replace_curly('{{lang|en|God Save the Queen}}{{en icon}}{{center|ファイル:United States Navy Band - God Save the Queen.ogg}}')

'God Save the QueenUnited States Navy Band - God Save the Queen.ogg'

In [159]:
fields_dict = {}
basic_info_pattern = '\|(.+)=(.+)'
for info_line in basic_info:
    # 強調マークアップを削除
    flat_line = re.sub("'?''", '', info_line)
    m = re.search(basic_info_pattern, flat_line)
    if m:
        field, val = m.groups()
        # 内部リンクマークアップを除去
        val = replace_internal_markup(val)
        # curly マークアップを除去
        val = replace_curly(val)
        fields_dict[field.strip()] = val.strip()
fields_dict

for _k in target_keys:
    print(_k, fields_dict[_k])

国章画像 イギリスの国章
国章リンク （国章）
標語 Dieu et mon droit<br />（フランス語:神と我が権利）
国歌 God Save the Queen<br />神よ女王を護り賜え<br />United States Navy Band - God Save the Queen.ogg
公用語 英語
首都 ロンドン（事実上）
元首等肩書 女王
元首等氏名 エリザベス2世
首相等肩書 首相
首相等氏名 ボリス・ジョンソン
他元首等肩書1 貴族院議長
他元首等氏名1 ノーマン・ファウラー
他元首等肩書2 庶民院議長
他元首等肩書3 最高裁判所長官
他元首等氏名3 ブレンダ・ヘイル
確立形態1 イングランド王国／スコットランド王国<br />（両国とも1707年合同法まで）
確立形態2 グレートブリテン王国成立<br />（1707年合同法）
確立形態3 グレートブリテン及びアイルランド連合王国成立<br />（1800年合同法）
通貨 UKポンド (£)
ccTLD .uk / .gb<ref>使用は.ukに比べ圧倒的少数。</ref>


## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい

In [169]:
import requests

S = requests.Session()

URL = "https://ja.wikipedia.org/w/api.php"
filename = "ファイル:Royal Coat of Arms of the United Kingdom.svg"

PARAMS = {
    "action": "query",
    "format": "json",
    "prop": "imageinfo",
    "titles": filename
}

DATA = S.get(url=URL, params=PARAMS).json()

PAGES = DATA["query"]["pages"]

for k, v in PAGES.items():
    print(v["title"] + " is uploaded by User:" + v["imageinfo"][0]["user"])

ファイル:Royal Coat of Arms of the United Kingdom.svg is uploaded by User:Pbrks


In [174]:
from IPython.display import Image
insert_filepath = filename.replace('ファイル:', '').replace(' ', '_')
target_file_url = f'https://commons.wikimedia.org/wiki/Special:FilePath/{insert_filepath}'
Image(url=target_file_url)